In [3]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import scrapy
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re,os
import pandas as pd
import numpy as np

In [210]:
url='https://www.filmaffinity.com/en/topgen.php?genre=TV_SE&country=&fromyear=&toyear=&nodoc&limit=100'

In [211]:
request=requests.get(url).content

In [212]:
soup = BeautifulSoup(request,"lxml")

In [257]:
counter=0
titlinfo=[]
rating=[]
for item in soup.find_all('ul'):
    
    a=item.find_all('li',{'class':'content'})
    b=item.find_all('div',{'class':'avg-rating'})
    c=item.find_all('div',{'class':'rat-count'})
    
    if a!=[] and b!=[] and c!=[]:
        counter+=1
        at=a[0].find('div',attrs={'class':'mc-title'}).get_text().replace(')','').split('(')
        ac=a[0].find('div',attrs={'class':'mc-title'}).select('img')[0].attrs['title']
        act=a[0].find('div',{'class':'credits'})
        if len(at)>3:
            titlinfo.append([at[0].strip(),at[-2].strip(),at[-1].strip(),ac.strip()])
        else:
            titlinfo.append([at[0].strip(),at[1].strip(),at[2].strip(),ac.strip()])
        avg_rating=b[0].get_text().strip()
        num_rat=c[0].get_text().strip().replace(',','')
        rating.append([avg_rating,num_rat])
        
filmaffinity_tv_series=pd.DataFrame(np.c_[titlinfo,rating],columns=['Title','Type','Year','Country','Avg Rating','Number of votes']).drop_duplicates(subset='Title').reset_index(drop=True)
filmaffinity_tv_series['Number of votes']=filmaffinity_tv_series['Number of votes'].astype('int64')
filmaffinity_tv_series['Avg Rating']=filmaffinity_tv_series['Avg Rating'].astype('float32')
filmaffinity_tv_series['Year']=filmaffinity_tv_series['Year'].astype('int32')
print(chr(27)+"[1;31m"+"Top %s TV Series for Filmaffinity Webpage: " %int(len(filmaffinity_tv_series)))
display(filmaffinity_tv_series.head(10))

Top 30 TV Series for Filmaffinity Webpage: 


,Title,Type,Year,Country,Avg Rating,Number of votes
0,The Wire,TV Series,2002,United States,8.9,44459
1,Breaking Bad,TV Series,2008,United States,8.8,95731
2,Game of Thrones,TV Series,2011,United States,8.6,105921
3,The Simpsons,TV Series,1989,United States,8.6,171127
4,Band of Brothers,TV Miniseries,2001,United States,8.5,54650
5,The Sopranos,TV Series,1999,United States,8.5,54389
6,True Detective,TV Series,2014,United States,8.5,56354
7,Rick and Morty,TV Series,2013,United States,8.4,21097
8,Monty Python's Flying Circus,TV Series,1969,United Kingdom,8.4,11902
9,Dekalog: The Ten Commandments,TV Miniseries,1988,Poland,8.4,1606


In [254]:
summary=filmaffinity_tv_series.groupby('Country').agg({'Avg Rating':'mean','Number of votes':'sum','Year':['min','max'],'Country':'count'})
summary.columns=summary.columns.droplevel()
summary.columns=['Avg Rating','Total Num of Votes','Year Min','Year Max','Count']
summary=summary.round({'Avg Rating': 2})
summary.sort_values('Total Num of Votes',ascending=False)

,Avg Rating,Total Num of Votes,Year Min,Year Max,Count
Country,,,,,
United States,8.39,810365,1989,2015,15
United Kingdom,8.30,57841,1969,2013,4
Japan,8.17,53408,2006,2011,6
Italy,8.30,7820,2008,2008,1
Poland,8.40,1606,1988,1988,1
Norway,8.10,735,2015,2015,1
Sweden,8.20,230,1973,1983,2


# IMDB

In [255]:
urlimdb='https://www.imdb.com/list/ls004729995/'

In [256]:
requesti=requests.get(urlimdb).content
soupi = BeautifulSoup(requesti,"lxml")

In [261]:
ap=soupi.find_all('div',{'class':'lister-item-content'})

In [287]:
title=[]
rating=[]
counter=0
for item in ap:
    if a!=[]:
        counter+=1
        a=item.select('a')[0].get_text().strip()
        b=item.find('span',{'class':'lister-item-year text-muted unbold'}).get_text().replace('(','').replace(')','').strip()
        c=item.find('span',{'class':'ipl-rating-star__rating'}).get_text()
        d=item.find('span',{'name':'nv'}).get_text()
        title.append([a,b])
        rating.append([c,d])
        if counter==30:
            break

In [292]:
imdb=pd.DataFrame(np.c_[title,rating],columns=['Title','Year','Avg Rating','Num of Votes'])
imdb.columns=['imdb.'+item for item in imdb.columns.tolist()]
display(imdb)

,imdb.Title,imdb.Year,imdb.Avg Rating,imdb.Num of Votes
0,Breaking Bad,2008–2013,9.5,"1,127,761"
1,The Wire (Bajo escucha),2002–2008,9.3,"233,035"
2,"Te quiero, Lucy",1951–1957,8.3,"20,143"
3,Seinfeld,1989–1998,8.9,"212,604"
4,El ala oeste de la Casa Blanca,1999–2006,8.7,"53,735"
5,The Oprah Winfrey Show,1986–2011,5.1,"4,270"
6,La hora de Bill Cosby,1984–1992,7.4,"28,204"
7,Los Soprano,1999–2007,9.2,"236,285"
8,The Tonight Show Starring Johnny Carson,1962–1992,8.4,"2,672"
9,60 Minutes,1968–,7.5,"2,588"


In [298]:
filmaffinity_tv_series_mod=filmaffinity_tv_series.copy()

In [299]:
comparing_top_30=pd.concat((filmaffinity_tv_series_mod,imdb),axis=1)
display(comparing_top_30.head(10))

,Title,Type,Year,Country,Avg Rating,Number of votes,imdb.Title,imdb.Year,imdb.Avg Rating,imdb.Num of Votes
0,The Wire,TV Series,2002,United States,8.9,44459,Breaking Bad,2008–2013,9.5,"1,127,761"
1,Breaking Bad,TV Series,2008,United States,8.8,95731,The Wire (Bajo escucha),2002–2008,9.3,"233,035"
2,Game of Thrones,TV Series,2011,United States,8.6,105921,"Te quiero, Lucy",1951–1957,8.3,"20,143"
3,The Simpsons,TV Series,1989,United States,8.6,171127,Seinfeld,1989–1998,8.9,"212,604"
4,Band of Brothers,TV Miniseries,2001,United States,8.5,54650,El ala oeste de la Casa Blanca,1999–2006,8.7,"53,735"
5,The Sopranos,TV Series,1999,United States,8.5,54389,The Oprah Winfrey Show,1986–2011,5.1,"4,270"
6,True Detective,TV Series,2014,United States,8.5,56354,La hora de Bill Cosby,1984–1992,7.4,"28,204"
7,Rick and Morty,TV Series,2013,United States,8.4,21097,Los Soprano,1999–2007,9.2,"236,285"
8,Monty Python's Flying Circus,TV Series,1969,United Kingdom,8.4,11902,The Tonight Show Starring Johnny Carson,1962–1992,8.4,"2,672"
9,Dekalog: The Ten Commandments,TV Miniseries,1988,Poland,8.4,1606,60 Minutes,1968–,7.5,"2,588"
